# Topic 1: Data Organization and Storage

<center>
<img src="images/structured_data.jpg" width="300">
</center>

## Data (science) is like an iceberg:

* Over 80% of time is spent cleaning/structuring data
* Over 80% of existing data is "unstructured"
* 90% of the data ever generated by humanity was generated in the last [two years](https://www.forbes.com/sites/bernardmarr/2018/05/21/how-much-data-do-we-create-every-day-the-mind-blowing-stats-everyone-should-read/#4f728b360ba9)!
    - ...but how much of it is useful?

<center>
<img src="images/data-iceberg.png" width="500">
</center>

In this course most data will be at least partially cleaned, but in the real world this is rarely the case.

## Establishing an efficient data pipeline

* Good integration of data storage + analysis can make or break a real project
* Separate raw data from analysis facilitates reproducibility and scalability
* Rapidly prototype and iteratevely improve.

Important concepts:
* Structured data
  - Schemas
  - Numpy array
  - Pandas dataframe
* Metadata
  - HDF5
* Unstructured data
  - JSON
  - APIs


## The data-structure continuum

Most structured

* **Index data with integers (matrices)**
* **Index data with strings/integers (dataframes)**
* Access data with structured queries (SQL)
* **Access data through web requests (APIs)**
* Access data through parsers (scraping)
* Access data with unstructured queries (search engines)

Least structured

## Definition of "structured data":

Datasets that can be defined by a well-determined organizational structure (schema).

* Matrices
* Spreadsheets (e.g. Excel) or "dataframes" (e.g. `R`, `pandas`)
* Relational databases (e.g. SQL)

**Schemas** enable efficient data storage by separating the context of the data from the data itself.

*Raw data:*

Car 1:
* Make: Toyota
* Model: Corolla
* Year: 2001

Car 2:
* Make: Toyota
* Model: Prius
* Year: 2012

Schema:
* Row = car number
* Columns = Make, Model, Year
* Values = [[Toyota, Corolla, 2001], [Toyota, Prius, 2012]]

Schema + Values = Raw Data

size(Schema + Values) < size(Raw Data)

### Advantages:

* fast/easy to query/manage
* intuitive to work with
* forces consideration of analysis ahead of time

### Disadvantages

* missing/inconsistent data (data fusion)
* balance of comprehensiveness vs. convenience in schema design
* analysis may be limited by data structure

There are numerous tools/strategies for storing structured data:

* Relational databases (structured query language, SQL) - Good for very large datasets (not covered in this course)
* Excel spreadsheets - common, but limited
* **Matrices** - common and intuitive but limited to numbers
* **Dataframes** - programmatic equivalent of spreadsheets. Implemented with `pandas` in Python. Can also function as "light" databases.
* **HDF5 files** - hierarchical files similar to directory structure. Extremely useful for huge array-like data structures due to "chunking". Several Python libraries, we will use `h5py`. 

## Matrices (Numpy arrays)

A Numpy array is the Python data type for storing/manipulating multi-dimensional matrices.

Matrices are an extreme example of structured data. Data is accessed by providing indices for each dimension. Indices must be integers, and all data must be numerical.

Matrices do not have a built-in schema system, so it must be managed separately. We will briefly explore how to work with matrices in Python and how to use them for data storage.

In [ ]:
import numpy as np #<- very common shorthand for numpy

## Creating `numpy` arrays

There are a number of ways to initialize new numpy arrays, for example from

* a Python list or tuples
* using functions that are dedicated to generating numpy arrays, such as `arange`, `linspace`, etc.
* reading data from files

In [ ]:
# a vector: the argument to the array function is a Python list
v = np.array([1,2,3,4])

print(v)

# a matrix: the argument to the array function is a nested Python list
M = np.array([[1, 2], [3, 4]])

print(M)

type(v), type(M)

The difference between the `v` and `M` arrays is only their shapes. We can get information about the shape and size of an array by using the `shape` and `size` properties.

In [ ]:
print(v.shape)
print(M.shape)
print(v.size)
print(M.size)

Arrays are similar to lists, but they must contain a single type:

In [ ]:
M[0,0] = "hello"

If we want, we can explicitly define the type of the array data when we create it, using the `dtype` keyword argument: 

In [ ]:
M = np.array([[1, 2], [3, 4]], dtype=complex)

M

### Creating arrays with functions

It is often more efficient to generate large arrays instead of creating them from lists. There are a few useful functions for this in numpy:

* `np.arange` - create a range with a specified step size (endpoints not included)
* `np.linspace` - create a range with a specified number of points (endpoints *are* included)
* `np.logspace` - create a range with a specified number of points in log space (endpoints *are* included)
* `np.mgrid` - create points on a multi-dimensional grid (similar to meshgrid in matlab)
* `np.random.rand` - create random number matrix from a uniform distribution
* `np.random.randn` - create random number matrix from a standard normal distribution
* `np.zeros` - create a matrix of zeros
* `np.ones` - create a matrix of ones
* `np.eye` - create identity matrix

In [ ]:
x = np.arange(0, 10, 0.5) # arguments: start, stop, step
print(x)
x = np.linspace(0,10,15)
print(x)
x = np.logspace(0,3,10,base=10)
print(x)
print([np.log10(xi) for xi in x])

In [ ]:
x, y = np.mgrid[0:5, 0:5] # similar to meshgrid in MATLAB
print(x)
print(y)

In [ ]:
# uniform random numbers in [0,1]
rand_uniform = np.random.rand(3,3)
print(rand_uniform)
# standard normal distributed random numbers
rand_normal = np.random.randn(3,3)
print(rand_normal)

In [ ]:
z = np.zeros((3,3)) #note that these take 1 tuple argument instead of multiple integers
one = np.ones((3,3))
I = np.eye(3,3) #but not this one... this is an annoying inconsistency.
print(z)
print(one)
print(I)

## File I/O

* Numpy has built-in functionality for reading/writing CSV or TSV (tab-separated value) files

In [ ]:
M = np.random.rand(6,6)
np.savetxt("random-matrix.csv", M)
M1 = np.genfromtxt("random-matrix.csv")
print(M1==M)

The ability to read in data from arbitrary text files is also useful. Consider the following `.dat` file that contains tab-separated values for temperature in Stockholm:

In [ ]:
!head datasets/stockholm_td_adj.dat #<- note that ! lets us run a bash command

In [ ]:
data = np.genfromtxt('datasets/stockholm_td_adj.dat')
print(data.shape)

## Question:

What is the schema for this dataset?

## Manipulating arrays

Once we generate `numpy` arrays, we need to interact with them. This involves a few operations:

* indexing - accessing certain elements
* index "slicing" - accessing certain subsets of elements
* fancy indexing - combinations of indexing and slicing

This is not very different from Matlab.

We can index elements in an array using square brackets and indices:

In [ ]:
# v is a vector, and has only one dimension, taking one index
print(v[0])
# M is a matrix, or a 2 dimensional array, taking two indices 
print(M[1,1])
# If an index is ommitted then the whole row is returned
print(M[1])
# This means that we can also index with multiple brackets if we want to type more:
print(M[1][1] == M[1,1])

The same thing can be achieved with using `:` instead of an index: 

In [ ]:
print(M[1,:]) # row 1
print(M[:,1]) # column 1

We can assign new values to elements or rows in an array using indexing:

In [ ]:
M[0,0] = 1
print(M)
M[:,2] = -1
print(M)

### Index slicing

Index slicing is the name for the syntax `M[lower:upper:step]` to extract a subset of an array.

In [ ]:
A = np.arange(1,20)
print(A)
print(A[1:8:2])
print(A[1:8]) #This is the most common usage
print(A[5:])
print(A[-3:])

Array values can also be assigned using slicing:

In [ ]:
A[1:3] = [-2,-3]
print(A)

Index slicing works exactly the same way for multidimensional arrays:

In [ ]:
R = np.random.rand(10,10,10)
print(R.shape)
subR = R[3:5, 1:4, 0]
print(subR.shape)
print(subR)

### Fancy indexing

Fancy indexing is the name for when an array or list is used in-place of an index: 

In [ ]:
R = np.random.rand(4,4)
print(R)
print('-'*10)
row_indices = [1, 3]
print(R[row_indices])

In [ ]:
col_indices = [1, -1] # remember, index -1 means the last element
print(R[row_indices, col_indices])

### Transposing arrays

Arrays can easily be transposed with `.T`.

In [ ]:
skinny = np.random.rand(8,2)
print(skinny)
print(skinny.shape)
fat = skinny.T
print(fat)
print(fat.shape)

These manipulations are extremely convenient - we only need a few characters to rapidly access/manipulate our data. However, remembering what each index means is tedious and error-prone. This can be included by baking the schema into the data structure.

## Pandas dataframes

Pandas dataframes are a very convenient way to interact with low-dimensional structured data. The basic dataframe object acts very similarly to an Excel file, but data can be manipulated with Python rather than clumsy Excel functions.

In [ ]:
%matplotlib inline
import pandas as pd

#Look at csv file - why skip 32 rows?
df = pd.read_csv('datasets/oceanic_data-brewer.csv',skiprows=32)
df['DATE']

Pandas dataframes are a cross between dictionaries and numpy arrays. Unlike arrays, they are allowed to hold multiple types, and they index columns and rows based on "keys" rather than numbers:

In [ ]:
#print(df.index) #row names
#print(df.columns) # column names
#print(df.values) #all data
print(df.dtypes)

It is easy to view and summarize data with the dataframe object:

In [ ]:
df.head(5)
#df.tail(10)
#df

Data can be accessed in several ways:

* Columns can be accessed directly with keys. 
* The `loc` method enables numpy-like indexing, fancy indexing, and slicing. 
* The `iloc` method is similar to `loc`, but indexes by position (rather than key)

These methods return `pandas.Series` objects, that are basically 1D dataframes.

In [ ]:
#df['LATITUDE']
#df.loc[1,'LATITUDE']
#df.loc[1]
#df.loc[:,['LATITUDE','LONGITUDE']]
#df.iloc[:,7]

Columns can be deleted in three ways:

* del : delete the Series from the dataframe
* pop() : delete the Series and return the Series
* drop(labels, axis) : return a new dataframe with Series removed (do not modify original df)

Rows must be "dropped".

**Pay attention to whether operations are "in place" or not**. Many `pandas` operations are *not* "in place" by default. This means that they return a copy of the dataframe with modifications, rather than modifying the original dataframe object. This can be very confusing.

In [ ]:
#Delete column
#times = df.pop('TIME')
#print(times)
#del df['EXPOCODE']
#dfnew = df.drop('CASTNO', axis=1)

#Delete row
df2 = df.drop(df.index[0])
df2 = df2.drop(df.index[-1])
df2.tail(3)

Let's simplify this data into only the columns that we care about:

In [ ]:
interesting = ['DATE','LATITUDE','LONGITUDE','DEPTH','SALNTY','OXYGEN','SILCAT','NITRAT','PHSPHT', 'ALKALI']
df = df[interesting]
df = df.drop(df.index[0])
df = df.drop(df.index[-1])
df

It looks like there are some artifacts since a concentration of -999.0 does not make sense. We can get rid of these using "boolean indexing":

In [ ]:
df.dtypes
df = df.astype('float')
df.dtypes
conc = ['OXYGEN','SILCAT','NITRAT','PHSPHT', 'ALKALI']
for var in conc:
    df = df[df[var] >= 0] #only take rows where concentration is >=0

df = df.reset_index() #<- THIS IS IMPORTANT!!!
df

This could also be achieved with OpenRefine. There is often no right way to do things in data science, so pick whatever feels easiest or most intuitive.

Note that the dates were not parsed correctly. We can fix this with a loop and assign the proper dates to the DATE column:

In [ ]:
def convert_DATE(date_col):
    dates = []
    for dt in date_col:
        dt = str(int(dt))
        y,m,d = dt[:4], dt[4:6], dt[6:]
        dates.append('-'.join([y,m,d]))
    print(len(dates))
    dates = pd.Series(dates)
    dates = pd.to_datetime(dates,yearfirst=True)
    return dates

dates = convert_DATE(df['DATE'])
df['DATE'] = dates # [(0, date0), (1, date1) ... ]

In [ ]:
df.head(10)

Pandas dataframes have some handy plotting features built in:

In [ ]:
print(conc)
#df[conc].plot()
df_sorted = df.sort_values('PHSPHT')
#df_sorted.plot(x='PHSPHT',y='SILCAT')
df_sorted.hist('SILCAT')

Pandas allows easily indexing by different columns:

In [ ]:
df2 = df.set_index('DATE')
df2['1981-06-03':'1981-09-07']

## Pandas multi-indexing

Pandas multi-indexing allows multi-dimensional `DataFrame` objects that act like different sheets/files in Excel. This is very useful for creating "mini databases" that allow organization of complex data with multiple dimensions.

Let's consider another dataset similar to the one we just worked with:

In [ ]:
new_df = pd.read_csv('datasets/oceanic_data-morliere.csv',skiprows=35)
#quickly clean up the data:
new_df = new_df[interesting]
new_df = new_df.drop(0)
new_df = new_df.astype('float')
for var in conc:
    new_df = new_df[new_df[var] > 0]
new_df = new_df.reset_index()
dates = convert_DATE(new_df['DATE'])
new_df['DATE'] = dates
new_df = new_df.set_index('DATE')
new_df

Now we might want to create a DateFrame that contains both sets of data, but index them by the scientist that collected the data (Brewer and Morliere). This can be achieved with the Multi Index functionality:

In [ ]:
brewer = df2
morliere = new_df
full = pd.concat({'Brewer':brewer, 'Morliere':morliere})
print(full.columns)
full = full.T
x = full['Brewer'].T['1981-05'] #<- get all measurements by Brewer in May 1981
print(x.columns)
x.head(10)

It is easy to convert from a `pandas` dataframe back to a `numpy` array (assuming all values are numerical):

In [ ]:
x.values

## Metadata provides additional context to datasets

* Metadata is "data about the data", and often contains details about where the dataset was collected or how it pertains to other similar datasets
    - Name of person recording the data
    - Place the data was recorded
    - Type of equipment used
* Data is often useless without its metadata

For example, we can think of a matrix/array of numbers as "data", and the definition of what a row/column means as "metadata". Without the definition of rows/columns the matrix has no meaning.

Keeping metadata and data synchronized is critical to ensuring that data remains valuable. This is why keeping a spreadsheet with column/row titles is a much more robust way of storing data than by keeping the matrix in one file and the column/row titles in another.

Metadata is particularly necessary in the case of extremely large datasets, since it enables efficient identification of relevant datasets through searches or databases of metadata.

## HDF 5 files

HDF5 files (hierarchical data format) are useful for structured data, especially for very large datasets. HDF5 files act like mini filesystems, and have "attributes" to include additional context.

We can create and manipulate HDF5 files from within Python using the `h5py` module.

In [ ]:
! rm test.hdf5

In [ ]:
import h5py
import numpy as np

f = h5py.File("test.hdf5", "w") #<- the "w" argument tells h5py to create a new file.
dset = f.create_dataset("dset", (100,100))
X = np.random.rand(100,100)

In [ ]:
#print(X[0,0])
#dset[:,:] = X
dset.shape
dset.name
dset[0,0]

HDF5 files can have groups so that multiple datasets can be stored/organized in a single file:

In [ ]:
#grp = f.create_group('random_data')
#d2 = grp.create_dataset('rand3D',(10,10,10))
R = np.random.randn(10,10,10)
d2[:,:,:] = R
d2.name

The HDF5 object directly interacts with the file, which can be opened with multiple instances.

In [ ]:
d2.shape
print(d2.name)
print(f['dset'][6,6])
f['dset'].shape
f['random_data/rand3D'].shape

#iteration goes through all datasets in the file
for d in f:
    print(d)

g = h5py.File('test.hdf5','r')
g['dset'][6,6] = 20

HDF5 files also support "attributes" that can tag datasets with additional info:

In [ ]:
f['dset'].attrs['name'] = 'random_matrix'
f['dset'].attrs['size'] = (100,100)
dset.attrs['name']

## Dealing with "big" data

Even Python cannot handle datasets of unlimited size. At some point, reading data in may become too slow to be practical, or impossible with RAM limits. Both `pandas` and `HDF5` have solutions for this:

* `pandas`: read data from large files in "chunks"
* `HDF5`: point to data on HDD and only read sub-sets when needed

## Exercise: Convert xyz csv file to HDF5 file

In [ ]:
rho = pd.read_csv("datasets/electron_density.csv")
rho.shape
rho.describe()

In [ ]:
for chunk in pd.read_csv("datasets/electron_density.csv",chunksize=10000):
    print(chunk.shape)

In [ ]:
! rm density.hdf5

In [ ]:
dx = 0.092826

rho['i'] = round(rho['x']/dx,0).astype(int)
rho['j'] = round(rho['y']/dx,0).astype(int)
rho['k'] = round(rho['z']/dx,0).astype(int)


xid = rho['i'].values
yid = rho['j'].values
zid = rho['k'].values

rhoval = rho['rho'].values
diffval = rho['difference'].values

size = (xid.max()+1,yid.max()+1,zid.max()+1)

rho_array = np.zeros(size)
diff_array = np.zeros(size)

size
rho_array.shape
rho_array.mean()

In [ ]:
#Naive solution takes forever!
print(rho.shape)
import time
t0 = time.time()
count = 0
for idx,row in rho.iterrows():
    count +=1
    i = int(row.i)
    j = int(row.j)
    k = int(row.k)
    rho_array[i,j,k] = row.rho
    diff_array[i,j,k] = row.difference
    t1 = time.time()
    if t1 > t0+5:
        break
print(i,j,k)
print("Total processed:",count)

In [ ]:
#Smart solution is almost instant!
rho_array[xid,yid,zid] = rhoval
diff_array[xid,yid,zid] = diffval


#Check to be sure it worked
xrand, yrand, zrand = np.random.randint(0,3,size=(3))
print(rho_array[xrand,yrand,zrand])

try:
    rho = rho.set_index(['i','j','k'])
except KeyError:
    pass #already ran this block

print(rho['rho'][int(xrand)][int(yrand)][int(zrand)])

In [ ]:
#Store in HDF5 file:
f = h5py.File('density.hdf5','w')
rho_hdf5 = f.create_dataset('rho',data=rho_array)
diff_hdf5 = f.create_dataset('diff',data=diff_array)
f.attrs['dx'] = dx
#rho_hdf5 = rho_array #<- this only assigns pointers! use the "data" keyword, or array.copy()
#diff_hdf5 = diff_array

In [ ]:
g = h5py.File('density.hdf5','r')
rho_2 = g['rho']
print(rho_array[4,5,6])
print(rho_2[4,5,6])
print(g.attrs['dx'])

## Definition of unstructured data

Data that does not have a clear pre-defined structure.

* Text documents
* Websites
* Videos
* Course documents

Definition is imprecise because "structure" may be implicit or hidden.

## Working with "unstructured" data

To analyze "unstructured" data you must impart some structure on it. Schema-free databases facilitate working with data with no (or ill-defined) structure.

#### Schema-free advantages

* flexible
* quick to set up
* easy to evolve/reconfigure

#### Schema-free disadvantages

* slow(er) to query
* harder to maintain
* some structure must still be defined

## Working with "unstructured" data?

#### Extracting "structure" from unstructured data
* text processing (natural language processing) - entire field of CS!
* data "scraping" - BeautifulSoup python package
* API's (technically not "unstructured")

Text processing and data scraping are beyond the scope of this class, but there are many tutorials online. API's (Application Programming Interface) are a common way of automatically accessing the structured version of "unstructured" data.

#### How to work with "unstructured" data
* JSON files - flexible "partially structured" data format
* schema-free "databases" (MongoDB, ElasticSearch)
* parsers for HTML/XML (e.g. BeautifulSoup)

We will not cover schema-free databases in lecture, but MongoDB is easy to set up and has a nice Python interface (`pymongo`). Most other interactions are essentially text-based ways of interacting with the internet.


## JSON Files

JavaScript Object Notation (JSON) files are a common way of adding structure to data so that it is easier to pass between code and interact with programatically. Although originally developed for JavaScript, JSON is now one of the most widespread file types and is supported by most programming languages.

JSON files are very intuitive to use with Python because they are basically just dictionaries and lists. 

In [ ]:
import json

info = '{"course":"ChBE 4803", "instructors": ["Medford", "Comer"], "size":45}' #<- note single/double quotes!
#info = str(info)
#print(info)
js_info = json.loads(info) #<- json.loads loads from a string, json.load loads from a file.

#js_info.keys()
#js_info['instructors']



#JSON is a great format for persistent storage of Python data structures:

with open('test.json','w') as f:
    json.dump(js_info, f)


In [ ]:
with open('test.json','r') as f:
    new_info = json.load(f)
    
new_info.keys()
new_info_dict = dict(new_info)
new_info_dict

## Example: PubChem database

[PubChem Search](https://pubchem.ncbi.nlm.nih.gov/)

* Extract and work with JSON representation
* Use RESTful API to access data programatically
* Demonstrate Python "wrapper" for the API

#### Goal 1: Extract SMILES representation, molecular weight, and boiling point from PubChem JSON file.

In [ ]:
import json

#Website: https://pubchem.ncbi.nlm.nih.gov/compound/222
#Download -> Data used to generate this page -> JSON -> Save

with open('datasets/ammonia.json') as f:
    nh3 = json.load(f)

In [ ]:
# Explore JSON structure
nh3['Record']['Section'][0]['Description']

Working with JSON data can be challenging if there are many nested structures, headers, etc. It is very useful to use a visualization tool:

* [JSON Viewer](http://jsonviewer.stack.hu/)
* [Code Beautify](https://codebeautify.org/jsonviewer)
* [Chrome Extension](https://chrome.google.com/webstore/detail/json-viewer/gbmdgpbipfallnflgajpaliibnhdgobh?hl=en-US)

From the visualizer we can see how to extract the information we need.

In [ ]:
SMILES = nh3['Record']['Section'][3]['Section'][2]['Section'][3]['Information'][0]['StringValue']
MW = nh3['Record']['Section'][4]['Section'][0]['Information'][0]['Table']['Row'][0]['Cell'][1]['NumValue']
BP = nh3['Record']['Section'][4]['Section'][1]['Section'][3]['Information'][2]['StringValue']
BP, C = BP.split('°')
SMILES

## What can go wrong here???

Converting unstructured information to structured is tedious! The goal and challenge is to not just do this once, but do it in a way that works for other inputs. This can be even more challenging:

In [ ]:
# https://pubchem.ncbi.nlm.nih.gov/compound/441203

with open('datasets/cisplatino.json') as f:
    cp = json.load(f)

In [ ]:
subsec = cp['Record']['Section']
for section in subsec:
    try:
        SMILES = section['Section'][2]['Section'][3]['Information'][0]['StringValue']
    except:
        pass
SMILES

In [ ]:
def section_by_name(sections, name):
    """ Take a list of Sections from PubChem JSON and return the section with a given name"""
    for s in sections:
        if s['TOCHeading'] == name:
            return s
        
section_by_name(cp['Record']['Section'], "Names and Identifiers")

We can use this new function to create a more robust way of extracting info from the PubChem JSON:

In [ ]:
def get_info(pc_json):
    """ Return the SMILES string, molecular weight, and boiling point from a PubChem JSON file"""
    info = {} #<- we can store the info in this string as we grab it
    ## Get SMILES string:
    namesec = section_by_name(pc_json['Record']['Section'], "Names and Identifiers")
    descsec = section_by_name(namesec['Section'], "Computed Descriptors")
    smilesec = section_by_name(descsec['Section'], "Canonical SMILES")
    SMILES = smilesec['Information'][0]['StringValue'] #<- we are assuming that there is only one entry here.
    info['SMILES'] = SMILES
    
    ## Get molecular weight
    propsec = section_by_name(pc_json['Record']['Section'],'Chemical and Physical Properties')
    compsec = section_by_name(propsec['Section'],'Computed Properties')
    MW = compsec['Information'][0]['Table']['Row'][0]['Cell'][1]['NumValue'] #<- we are assuming the table has a fixed structure
    info['molecular_weight'] = MW
    
    ## Get boiling point
    
    ### boiling point is in the same properties section as molecular weight, so start from there
    expsec = section_by_name(propsec['Section'],"Experimental Properties")
    try:
        bpsec = section_by_name(expsec['Section'], "Boiling Point")
        bpstring = bpsec['Information'][2]['StringValue'] #<- two problems!
        info['boiling'] = bpstring
    except TypeError:
        pass
    
    ## discuss how to handle problems
    
    return(info)
    
info = get_info(cp)
print(info)

Even with semi-structured data (JSON), it can be challenging to robustly and reliably extract structured information for analysis!

## API's (Application Programming Interfaces)

API's are like GUI's for experts. They are not limited to "unstructured" data, or even data in general. API is a term for any programmatic structure that makes it easier to interact with a more complex underlying code or data structure. However, they are particularly prevalent in data science because accessing data is much less painful.

## RESTful API's

REST stands for "representational state transfer", and is a protocol that enables accessing data directly through a URL. This is a very common and very powerful approach because it allows the data provider to abstract the database back-end from the API. In other words, data providers can provide a uniform interface to data in relational (schema-driven) databases, schema-free databases, file servers, or services in any programming language. All the user needs to know is how to "query" from a URL. If you pay attention to URL's as you browse the web you will see that you use RESTful API's all the time without knowing it!

<center>
<img src="images/RESTful.png" width="500">
</center>

RESTful API's are simple enough that you can use them without specialized libraries. You just need to use HTTP protocol, which is implemented in the `requests` Python library:

In [ ]:
import requests

response = requests.get("http://www.chbe.gatech.edu/")
response.text

RESTful API's are designed to return data in specific structures, and respond to specific queries that are embedded in the URL. A few notes:

* Many API's require a "key" or "token". This is to avoid spammers overloading their servers.
* Most API's also limit the amount of data per request, and the rate of requests.
* It is still necessary to understand the underlying structure of the data you are querying.

You should always start by reading the documentation of an API to learn what you can/can't do.

#### Goal 2: Use PubChem RESTful API to automatically get SMILES representation of a given compound

[PubChem API tutorial documentation](http://pubchemdocs.ncbi.nlm.nih.gov/pug-rest-tutorial$_Toc458584421)

[PubChem API full documentation](http://pubchemdocs.ncbi.nlm.nih.gov/pug-rest)

Let's start by seeing if we can get the "search" part to work

In [ ]:
r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/ammonia/cids/TXT')
r.text #<- this is the CID of the compound  

In [ ]:
def get_CID(chemical):
    r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{}/cids/TXT'.format(chemical))
    return r.text

cid = get_CID('ethanol')
print(cid)

Now we need to understand the structure of the query to decide how to search. From the documentation:

* prolog: `https://pubchem.ncbi.nlm.nih.gov/rest/pug`

* input: `/compound/name/ammonia`

* operation: `/cids`

* output: `/TXT`

We already have the input operation working, and since we just want SMILES the output can also be TXT. We just need to modify the operation.

In [ ]:
def get_SMILES(chemical):
    r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{}/property/CanonicalSMILES/TXT'.format(chemical))
    return r.text

N = get_SMILES('ethylene glycol')
print(N)

This is much easier, less memory intensive, and more robust, than trying to extract the property from the full output! However, if you do really want to parse from the full output you can do that too:

In [ ]:
def get_full(chemical):
    r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{}/record/json'.format(chemical))
    return r.text

json_string = get_full('ammonia')
nh3 = json.loads(json_string)
nh3['PC_Compounds'] #<- Note that this JSON is in a very different structure from the original!

## Python API's

RESTful API's are widely used and easy to interact with. However, reading the documentation and converting more complex queries into the proper URL can be tedious and time consuming. Furthermore, not all data sources use RESTful API's.

Python is one of the most common languages for API's, and widely-used data sources (e.g. PubChem) will often have a Python "wrapper" for their RESTful API.

We can use the [PubChemPy](https://pypi.python.org/pypi/PubChemPy/1.0) API to achieve the same goal, but we will need to install it first:

`pip install PubChemPy`

In [ ]:
import pubchempy as pcp
#Now we have access to some more intuitive function names and documentation
help(pcp)
#dir(pcp)

Python APIs make code more readable, and are more intuitive to learn:

In [ ]:
compounds = pcp.get_compounds('Ammonia','name')
nh3 = compounds[0]
nh3.atoms #<- the full .json output is already parsed into a nice Python data structure
nh3.bonds

We can access the same SMILES string via the Python API:

In [ ]:
dir(nh3) #<- the Python API doesn't store the SMILES string by default
None

In [ ]:
p = pcp.get_properties('CanonicalSMILES', 'ammonia', 'name')
print(p) #<- this works, but is it really better than the RESTful version?

## Conclusions

* Structured data is efficient and convenient, but rigid. 
    - All the work goes into setting up the schema and ensuring that only good data gets in.
    - Structured data is valuable but rare
* Metadata provides context for structured data
    - Separating the context from the data enables rapid searches, especially for large datasets
    - Metadata can also be "structured" or "unstructured"
    - Keeping metadata and data "synchronized" is facilitated by file types that integrate meta-data with data.
* Unstructured data is flexible but more complex to query. 
    - It is easy to put data in regardless of whether it is good or not, but you have to "clean" it after querying
    - Most data generated by others is available only as "unstructured" data
    - Unstructured data can be "structured" manually or by using API's

When retrieving data it is a good idea to read about all of the available retrieval strategies (web scraping, direct download, RESTful API's, Python API's) and design a strategy that maximizes efficiency and flexibility.

When storing your own data you should find a balance between "unstructured" and "structured" that makes sense based on your project. Consider setting up a (schema-free) database and/or custom API to create a seamless interface between your data source and your analysis code, and/or using file types like HDF5 that keep meta-data and data together.

## Further Reading:

* [Official pandas tutorials and cookbooks](https://pandas.pydata.org/pandas-docs/stable/tutorials.html)
* [10 minutes to pandas tutorial](https://pandas.pydata.org/pandas-docs/stable/10min.html)
* [Usefule Pandas features](http://nbviewer.jupyter.org/urls/gist.github.com/wesm/4757075/raw/a72d3450ad4924d0e74fb57c9f62d1d895ea4574/PandasTour.ipynb)
* [DataCamp Pandas tutorial](https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python)
* [Visualization pandas tutorial](https://pandas.pydata.org/pandas-docs/stable/visualization.html)
* [Multi-index pandas tutorial](https://pandas.pydata.org/pandas-docs/stable/advanced.html)
* [HDF5 quick start](http://docs.h5py.org/en/latest/quick.html)
* [Hitchhiker's Guide to Python JSON tutorial](http://docs.python-guide.org/en/latest/scenarios/json/)
* [RESTful details](https://restfulapi.net/)
* [PubChem RESTful API tutorial](http://pubchemdocs.ncbi.nlm.nih.gov/pug-rest-tutorial)
* [PubChem Python API documentation](https://pypi.python.org/pypi/PubChemPy/1.0)